In [1]:
#Experimental Design
#Choose SINGLE optimal concentration of each additive from the combinatoric experiments
#Combine these additives singly AND combinatorically with the following fuel agumentations:
#Fuel Sources:
#   Energy Buffer[NTPs, 3GPA, tRNAs etc.] 
#   Amino Acid Solution [Amino Acids]
#   RNA-polymerase Buffer [Source of NTPs]
#   +CTRL GFP [Is DNA a limiting factor?]

import murraylab_tools.echo as mt_echo
import os.path
from scipy.special import comb as nCk
from itertools import combinations
import numpy as np

n_additives = 5

#Additives
conc_vitC = 1.0 #STOCK
conc_vitE = 1.0 #STOCK
conc_superoxide_dismutase = 1.0 #1/10 STOCK + 9/10 H20 (AU)
conc_pyrophosphatase = 1.0 #1/10 STOCK + 9/10 H20 (AU)
conc_polyphosphate_kinase = 1.0 #1/10 STOCK + 9/10 H20 (AU)

output_name = "EnergyAndFuelAdditives"
plate_file = "SourcePlate.dat"
rxn_vol = 10000 #nL

#Concentrations to use for each additive
additive_concentration_dict = {
    "superoxide dismutase": .05,
    "pyrophosphatase": .02,
    "vitamin_C": .01,
    "vitamin_E":  .05, 
    "polyphosphate kinase": .01
}

print("additive volumes:", [str(a)+": "+str(np.array(additive_concentration_dict[a])*rxn_vol)+"nL" for a in additive_concentration_dict])

K_stock = 3000 #mM
K_conc = 80 #mM
Mg_stock = 100 #mM
Mg_conc = 10 #mM
posCTRL_stock = 40 #nM
posCTRL_conc = 1.5 #nM

#Master Mix Recipe
AA_frac = .125 #Amino Acids per rxn vol
E_frac = 0.075 #Energy MM per rxn vol
PEG_frac = .05 #PEG per rxn vol
Buffer_frac = AA_frac+E_frac+PEG_frac
Extract_frac = .33
MM_padding = .1 #Fraction extra master mix to make
Mg_frac = Mg_conc/Mg_stock
K_frac = K_conc/K_stock
Salt_frac = K_frac+Mg_frac

MM_frac = (Buffer_frac+Extract_frac+Salt_frac)
used_rxn_vol = rxn_vol*MM_frac
used_rxn_vol = used_rxn_vol - (used_rxn_vol % 25) #Rounds down to avoid 25 nL increment errors
left_volume = (rxn_vol - used_rxn_vol)
left_frac = (rxn_vol - used_rxn_vol)/rxn_vol
print("leftover volume:", left_volume/1000, "leftover frac:", (rxn_vol - used_rxn_vol)/rxn_vol)

#How much extra of each energy component to add (in fractions of reaction volume)
energy_concentration_dict = {
    "pos ctrl gfp":[0, posCTRL_conc],
    "energy buffer":[E_frac*.25, E_frac*.5],
    "amino acids":[AA_frac*.25, AA_frac*.5],
    "ntps":[E_frac*.25, E_frac*.5],
}

total_energy_conditions = sum([len(energy_concentration_dict[e]) for e in energy_concentration_dict])
total_energy_conditions += 2 #Adding conditions for (ntp + amino acids) and (energy buffer + amino acids)

print("Energy source volumes:", [(e, np.array(energy_concentration_dict[e])*rxn_vol) for e in energy_concentration_dict])
rxn_count0 = nCk(n_additives, 0)
rxn_count1 = nCk(n_additives, 1)
rxn_count2 = nCk(n_additives, 2)

total_additive_combinations = rxn_count0 + rxn_count1 + rxn_count2# + rxn_count3 + rxn_count4
total_reactions = total_additive_combinations*total_energy_conditions

total_volume = total_reactions*rxn_vol
print("Total Additive Combinations:",total_additive_combinations, "Total Energy Condtions:", total_energy_conditions, "Total Reactions:", total_reactions)
print("total combinations")
print("\tCombinations with 0 additives:", rxn_count0, "=", rxn_count0*total_additive_combinations, "reactions")
print("\tCombinations with 1 additive:", rxn_count1, "=", rxn_count1*total_additive_combinations, "reactions")
print("\tCombinations with 2 additives:", rxn_count2, "=", rxn_count2*total_additive_combinations, "reactions")

MM_txt = "Mix the following MM: \n"
MM_txt += "\tExtract: "+str((1+MM_padding)*Extract_frac*total_volume/1000)+"uL\n"
MM_txt += "\tAmino Acids: "+str((1+MM_padding)*AA_frac*total_volume/1000)+"uL\n"
MM_txt += "\tEnergy Buffer: "+str((1+MM_padding)*E_frac*total_volume/1000)+"uL\n"
MM_txt += "\tPEG: "+str((1+MM_padding)*PEG_frac*total_volume/1000)+"uL\n"
MM_txt += "\tMG-glut: "+str((1+MM_padding)*Mg_frac*total_volume/1000)+"uL\n"
MM_txt += "\tK-glut: "+str((1+MM_padding)*K_frac*total_volume/1000)+"uL\n"
MM_txt += "\nAdd "+str(used_rxn_vol/1000)+"uL of MM to wells:"

starting_well_txtl = "G11"

well_definitions = {}
wells = []
def increment_well(well):
    if well not in wells:
        wells.append(well)
    row = well[0]
    col = int(well[1:])
    if col == 24:
        col = "8"
        row = str(chr(ord(row) + 1))
    else:
        col = str(col+1)
    return row+col

well_txtl = starting_well_txtl
for i in range(int(total_reactions)-1):
    well_txtl = increment_well(well_txtl)
    
MM_txt+=starting_well_txtl+"..."+well_txtl+"\n"

f = open(output_name+" Master Mix Instructions.txt", 'w')
f.write(MM_txt)
f.close()
print(MM_txt)

additive volumes: ['superoxide dismutase: 500.0nL', 'pyrophosphatase: 200.0nL', 'vitamin_C: 100.0nL', 'vitamin_E: 500.0nL', 'polyphosphate kinase: 100.0nL']
leftover volume: 2.95 leftover frac: 0.295
Energy source volumes: [('pos ctrl gfp', array([    0., 15000.])), ('energy buffer', array([187.5, 375. ])), ('amino acids', array([312.5, 625. ])), ('ntps', array([187.5, 375. ]))]
Total Additive Combinations: 16.0 Total Energy Condtions: 10 Total Reactions: 160.0
total combinations
	Combinations with 0 additives: 1.0 = 16.0 reactions
	Combinations with 1 additive: 5.0 = 80.0 reactions
	Combinations with 2 additives: 10.0 = 160.0 reactions
Mix the following MM: 
	Extract: 580.8000000000001uL
	Amino Acids: 220.00000000000003uL
	Energy Buffer: 132.0uL
	PEG: 88.00000000000001uL
	MG-glut: 176.00000000000003uL
	K-glut: 46.93333333333334uL

Add 7.05uL of MM to wells:G11...P17



In [2]:
source_plate = mt_echo.SourcePlate(filename = plate_file)
echo_vol = rxn_vol-used_rxn_vol
echo_calc = mt_echo.EchoRun(plate = source_plate, rxn_vol = rxn_vol)

water = mt_echo.EchoSourceMaterial('water', 1.0, 0.0, source_plate)
MM = mt_echo.EchoSourceMaterial("MM", 1.0, 0.0, source_plate)
superoxide_dismutase = mt_echo.EchoSourceMaterial('superoxide dismutase', conc_superoxide_dismutase, 0.0, source_plate)
pyrophosphatase = mt_echo.EchoSourceMaterial('pyrophosphatase', conc_pyrophosphatase, 0.0, source_plate)
polyphosphate_kinase = mt_echo.EchoSourceMaterial('polyphosphate kinase', conc_polyphosphate_kinase, 0.0, source_plate)
vitC = mt_echo.EchoSourceMaterial('vitamin_C', conc_vitC, 0, source_plate)
vitE = mt_echo.EchoSourceMaterial('vitamin_E', conc_vitE, 0, source_plate)

additives = [superoxide_dismutase, pyrophosphatase, vitC, vitE, polyphosphate_kinase]
additive_combinations = list(combinations(additives, 0))+list(combinations(additives, 1)) + list(combinations(additives, 2))

pos_ctrl_gfp = mt_echo.EchoSourceMaterial('pos ctrl gfp', posCTRL_stock, 0, source_plate)
energy_buffer = mt_echo.EchoSourceMaterial("energy buffer", 1.0, 0, source_plate)
amino_acids = mt_echo.EchoSourceMaterial("amino acids", 1.0, 0, source_plate)
ntps = mt_echo.EchoSourceMaterial("ntps", 1.0, 0, source_plate)

energy_additives_list = [[pos_ctrl_gfp], [energy_buffer], [amino_acids], [ntps], [energy_buffer, amino_acids], [ntps, amino_acids]]

well_txtl = starting_well_txtl

for additives in additive_combinations: #Cycle through additive combinations
    for energy_additives in energy_additives_list: #cycle through energy additions
        if len(energy_additives) == 1:
            energy_additive = energy_additives[0]
            energy_additive2 = None
            N_e_concs = 2 #concentrations per energy
        else:
            energy_additive = energy_additives[0]
            energy_additive2 = energy_additives[1]
            N_e_concs = 1 #fewer concentrations for the pairs
            
        for conc_ind in range(N_e_concs): #cycle through concentrations
            e_conc = energy_concentration_dict[energy_additive.name][conc_ind]
            
            well_definitions[well_txtl] = []
            echo_calc.add_material_to_well(MM, used_rxn_vol/rxn_vol, well_txtl, pipette_by_hand = True)
            
            if energy_additive.name == 'pos ctrl gfp':
                echo_calc.add_material_to_well(energy_additive, e_conc+posCTRL_conc, well_txtl) #Add energy Additive (Which could be +ctrl)
                well_definitions[well_txtl] = [(energy_additive.name, e_conc+posCTRL_conc)]
            else:
                echo_calc.add_material_to_well(pos_ctrl_gfp, posCTRL_conc, well_txtl) #Add +CTRL
                echo_calc.add_material_to_well(energy_additive, e_conc, well_txtl)
                well_definitions[well_txtl] = [('pos ctrl gfp', posCTRL_conc), (energy_additive.name, e_conc)]
            
            if energy_additive2 != None:
                e_conc2 = energy_concentration_dict[energy_additive2.name][conc_ind]
                echo_calc.add_material_to_well(energy_additive2, e_conc2, well_txtl)
                well_definitions[well_txtl] += [(energy_additive2.name, e_conc2)]
                
            for additive in list(additives):
                conc = additive_concentration_dict[additive.name]
                echo_calc.add_material_to_well(additive, conc, well_txtl)
                well_definitions[well_txtl] += [(additive.name, conc)]

            well_txtl = increment_well(well_txtl)
    
        
echo_calc.fill_all_wells_with(water)
echo_calc.write_picklist(output_name)

well_def_txt = "well\ttab seperated additives\n"
for well in well_definitions:
    well_def_txt += well
    for (a, c) in well_definitions[well]:
        well_def_txt += "\t"+a+"\t"+str(c)
    well_def_txt+="\n"

f = open(output_name+"_well_definitions.txt", 'w')
f.write(well_def_txt)
f.close()

In [3]:
output_name

'EnergyAndFuelAdditives'